# Projeto de Machine Learning (TP1) - Predição de Preços de Carros

Este notebook implementa um modelo de machine learning para prever preços de carros usados baseado em suas características.

## 1. Instalação de Dependências {#instalacao}

Primeiro, vamos instalar todas as bibliotecas necessárias para o projeto.

In [ ]:
# Instalação das dependências (executar apenas uma vez)
%pip install --upgrade pandas

## 2. Carregamento e Exploração dos Dados {#exploracao}

Agora vamos carregar o dataset e fazer uma análise exploratória inicial para entender os dados com que estamos trabalhando.

In [ ]:
# Importar bibliotecas e carregar dados
import pandas as pd

# Carregar o dataset (versão reduzida para performance)
df = pd.read_csv("train_small.csv")

print(f"Dataset carregado com {len(df)} registros")
df.info()

## 3. Preparação dos Dados {#preparacao}

Agora vamos preparar os dados para o modelo, separando features (X) e target (y).

In [ ]:
X = df.drop(columns=["price"])
Y = df["price"]

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# separar colunas por tipo de dados
categorical_cols = X.select_dtypes(include=["object"]).columns  # colunas de texto/categoria
numeric_cols = X.select_dtypes(exclude=["object"]).columns      # colunas numéricas

# criar preprocessador para tratar diferentes tipos de dados
preprocessor = ColumnTransformer(
    transformers=[
        ("categorias", OneHotEncoder(handle_unknown="ignore"), categorical_cols),  # converte categorias em 0s e 1s
        ("numericas", "passthrough", numeric_cols)                                # mantem numeros como estao
    ]
)

# pipeline completo: preprocessamento / modelo
model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regressor", LinearRegression())
    ]
)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Dividir dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_val)

# Calcular erro (RMSE)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f"RMSE do modelo baseline: ${rmse:,.2f}\n")

print("Distribuição do preço (target):")
print(f"Preço médio:   ${Y.mean():,.2f}")
print(f"Mediana:       ${Y.median():,.2f}")
print(f"25% percentil: ${Y.quantile(0.25):,.2f}")
print(f"75% percentil: ${Y.quantile(0.75):,.2f}")
print(f"Mínimo:        ${Y.min():,.2f}")
print(f"Máximo:        ${Y.max():,.2f}")

## Pipeline KNN + Normalizaçao

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

knn_model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),   # mesmo One-Hot + num
        ("scaler", StandardScaler(with_mean=False)),
        ("knn", KNeighborsRegressor())
    ]
)


## 6. Otimizações de Performance

Para melhorar a performance do modelo e reduzir o tempo de processamento, foram feitas as seguintes otimizações:

- **Dataset reduzido**: Usando uma amostra de 15.000 registros (vs 188.533 originais)
- **Grid de parâmetros simplificado**: Reduzindo o número de valores testados
- **Cross-validation reduzida**: Usando 3 folds ao invés de 5
- **Processamento paralelo**: Utilizando todos os cores disponíveis (`n_jobs=-1`)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(
    knn_model,
    X,
    Y,
    cv=5,
    scoring="neg_root_mean_squared_error"
)

rmse_scores = -scores

print("RMSE por fold:", rmse_scores)
print(f"RMSE médio: {rmse_scores.mean():,.2f}")

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    "knn__n_neighbors": [3, 5, 7],
    "knn__weights": ["uniform", "distance"],
    "knn__p": [1, 2]
}

grid_search = GridSearchCV(
    knn_model,
    param_grid,
    cv=3,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=1
)

print("Iniciando GridSearchCV com dataset reduzido...")
grid_search.fit(X, Y)

print("Melhor RMSE:", -grid_search.best_score_)
print("Melhores parâmetros:", grid_search.best_params_)
